<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231122-%E6%B5%8B%E8%AF%95python%E7%89%88%E5%9B%9B%E9%98%B6%E8%B0%90%E6%B3%A2%E7%9A%84%E6%AD%A3%E7%A1%AE%E6%80%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-22').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1245


In [5]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [10]:
# 二阶谐波
harmonicIndependents_4 = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2', 'cos3', 'sin3', 'cos4', 'sin4'])

def add_sincos_4(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  timeRadians3 = image.select('t').multiply(6 * math.pi)
  timeRadians4 = image.select('t').multiply(8 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2')).addBands(timeRadians3.cos().rename('cos3')).addBands(timeRadians3.sin().rename('sin3')).addBands(timeRadians4.cos().rename('cos4')).addBands(timeRadians4.sin().rename('sin4'))

harmonicSentinel_4 = dataset.map(add_sincos_4)

harmonicTrend_4 = harmonicSentinel_4.select(harmonicIndependents_4.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_4.length(),numY=1))

harmonicTrendCoefficients_4 = harmonicTrend_4.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_4])

def fit_4(image):
  return image.addBands(image.select(harmonicIndependents_4).multiply(harmonicTrendCoefficients_4).reduce('sum').rename('fitted'))

fittedHarmonic_4 = harmonicSentinel_4.map(fit_4)

In [11]:
point = ee.Geometry.Point(103.69442853431933, 30.551562081391808)

rTable = ee.FeatureCollection(fittedHarmonic_4.filterBounds(point).map(lambda image: ee.Feature(
    None,
    {
        'date': image.date().format('yyyy-MM-dd'),
        'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
        'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
    }
)))

rTable_dict = rTable.getInfo()

dates = [feat['properties']['date'] for feat in rTable_dict['features']]
ndvi_values = [feat['properties']['NDVI'] for feat in rTable_dict['features']]
fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

df = pd.DataFrame({'Date': dates, 'NDVI': ndvi_values, 'Fitted': fitted_values})

df = df.dropna()

print(df)

           Date      NDVI    Fitted
0    2022-01-01  0.480345  0.630368
11   2022-01-29  0.638256  0.576085
17   2022-02-13  0.727610  0.602311
25   2022-03-05  0.589392  0.694511
26   2022-03-07  0.687620  0.701744
27   2022-03-10  0.715548  0.710381
29   2022-03-15  0.706660  0.717214
30   2022-03-17  0.713792  0.716877
39   2022-04-09  0.631566  0.570115
43   2022-04-19  0.521598  0.447304
44   2022-04-21  0.573421  0.422154
46   2022-04-26  0.480552  0.361412
49   2022-05-04  0.220395  0.279772
51   2022-05-09  0.178203  0.244097
54   2022-05-16  0.197687  0.219465
71   2022-06-28  0.618742  0.593569
74   2022-07-05  0.619651  0.666791
75   2022-07-08  0.668581  0.693438
81   2022-07-23  0.735616  0.774721
82   2022-07-25  0.686726  0.779002
83   2022-07-28  0.771458  0.782778
87   2022-08-07  0.763696  0.774587
88   2022-08-09  0.751144  0.769632
89   2022-08-12  0.808487  0.760363
93   2022-08-22  0.608793  0.716237
99   2022-09-06  0.694187  0.620493
117  2022-10-21  0.278708  0